In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
import concurrent.futures as cf
import multiprocessing as mp

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

In [ ]:
site = 'https://stats.stackexchange.com/questions?tab=Votes'
noofpages = []
response = requests.get(site)
soup = BeautifulSoup(response.text, 'html.parser')
noofpages = int(soup.find('div', class_= 's-pagination site65 themed pager float-left').find_all('a')[-2].text)

In [ ]:
noofpages

In [ ]:
def scrape():
    question_links = []
    for i in tqdm(range(1,int(noofpages)+1)):
        if i == 1:
            url = 'https://stats.stackexchange.com/questions?tab=Votes'
        else:
            url = f'https://stats.stackexchange.com/questions?tab=Votes&page={i}'
        page = requests.get(url,headers=headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        for link in soup.find('div', attrs= {'class':'flush-left','id':'questions'}).find_all('a', attrs = {'class':'s-link'}):
            question_links.append(link.get('href'))
    return pd.DataFrame(question_links,columns=['Question Links'])

In [ ]:
def scrape_page(url):
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    return [link.get('href') for link in soup.find('div', attrs={'class': 'flush-left', 'id': 'questions'}).find_all('a', attrs={'class': 's-link'})]

def scrape(noofpages):
    question_links = []
    urls = []
    base_url = 'https://stats.stackexchange.com/questions?tab=Votes'
    
    with cf.ThreadPoolExecutor(max_workers=mp.cpu_count()) as executor:
        for i in range(1, int(noofpages) + 1):
            if i == 1:
                urls.append(base_url.format(''))
            else:
                urls.append(base_url.format(f'&page={i}'))
        
        # Submit scraping tasks to the ThreadPoolExecutor
        future_to_url = {executor.submit(scrape_page, url): url for url in urls}
        
        # Process the completed tasks
        for future in cf.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                result = future.result()
                question_links.extend(result)
            except Exception as e:
                print(f"Error while scraping {url}: {e}")

    return pd.DataFrame(question_links, columns=['Question Links'])


In [ ]:
# Replace this with the number of pages you want to scrape
df = scrape(5)
print(df)
